In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T

spark = SparkSession.builder.appName('leadlag').getOrCreate()

23/12/27 10:46:01 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [81]:
product_data = [
(1,"iphone","01-01-2023",1500000),
(2,"samsung","01-01-2023",1100000),
(3,"oneplus","01-01-2023",1100000),
(1,"iphone","01-02-2023",1300000),
(2,"samsung","01-02-2023",1120000),
(3,"oneplus","01-02-2023",1120000),
(1,"iphone","01-03-2023",1600000),
(2,"samsung","01-03-2023",1080000),
(3,"oneplus","01-03-2023",1160000),
(1,"iphone","01-04-2023",1700000),
(2,"samsung","01-04-2023",1800000),
(3,"oneplus","01-04-2023",1170000),
(1,"iphone","01-05-2023",1200000),
(2,"samsung","01-05-2023",980000),
(3,"oneplus","01-05-2023",1175000),
(1,"iphone","01-06-2023",1100000),
(2,"samsung","01-06-2023",1100000),
(3,"oneplus","01-06-2023",1200000)
]


from pyspark.sql.functions import *

emp_data = [(1,'manish',50000,'IT','m'),
(2,'vikash',60000,'sales','m'),
(3,'raushan',70000,'marketing','m'),
(4,'mukesh',80000,'IT','m'),
(5,'priti',90000,'sales','f'),
(6,'nikita',45000,'marketing','f'),
(7,'ragini',55000,'marketing','f'),
(8,'rashi',100000,'IT','f'),
(9,'aditya',65000,'IT','m'),
(10,'rahul',50000,'marketing','m'),
(11,'rakhi',50000,'IT','f'),
(12,'akhilesh',90000,'sales','m')]

emp_schema=['id','name','salary','dept','gender']

emp_df = spark.createDataFrame(data=emp_data,schema=emp_schema)
emp_df = emp_df.select('id','name','salary','gender','dept')
#emp_df.show()

product_schema = ["product_id", "product_name", "sales_date", "sales"]
product_df = spark.createDataFrame(data=product_data, schema=product_schema)\
                    .withColumn('sales_date', F.expr("to_date(sales_date, 'dd-MM-yyyy')"))\
                    #.withColumn('previous_sales', F.expr('lag(sales) over(partition by product_id order by sales_date desc)'))\
                    #.withColumn('profit%', F.expr("cast ((sales-previous_sales)/previous_sales as decimal(10, 2))"))
                    
product_df.sort(F.expr("product_id").asc()).show()

+----------+------------+----------+-------+
|product_id|product_name|sales_date|  sales|
+----------+------------+----------+-------+
|         1|      iphone|2023-04-01|1700000|
|         1|      iphone|2023-05-01|1200000|
|         1|      iphone|2023-01-01|1500000|
|         1|      iphone|2023-02-01|1300000|
|         1|      iphone|2023-06-01|1100000|
|         1|      iphone|2023-03-01|1600000|
|         2|     samsung|2023-06-01|1100000|
|         2|     samsung|2023-05-01| 980000|
|         2|     samsung|2023-02-01|1120000|
|         2|     samsung|2023-01-01|1100000|
|         2|     samsung|2023-04-01|1800000|
|         2|     samsung|2023-03-01|1080000|
|         3|     oneplus|2023-03-01|1160000|
|         3|     oneplus|2023-04-01|1170000|
|         3|     oneplus|2023-06-01|1200000|
|         3|     oneplus|2023-05-01|1175000|
|         3|     oneplus|2023-02-01|1120000|
|         3|     oneplus|2023-01-01|1100000|
+----------+------------+----------+-------+



In [41]:
dff = spark.range(10).select('*', F.expr("add_months(to_date('2023-12-01'), -1*id) dts"))
dff.withColumn('lg', F.expr("lead(dts, 1) over(order by dts)")).orderBy('id').show()

23/12/27 12:04:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/27 12:04:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/27 12:04:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/27 12:04:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/27 12:04:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/12/27 12:04:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---+----------+----------+
| id|       dts|        lg|
+---+----------+----------+
|  0|2023-12-01|      null|
|  1|2023-11-01|2023-12-01|
|  2|2023-10-01|2023-11-01|
|  3|2023-09-01|2023-10-01|
|  4|2023-08-01|2023-09-01|
|  5|2023-07-01|2023-08-01|
|  6|2023-06-01|2023-07-01|
|  7|2023-05-01|2023-06-01|
|  8|2023-04-01|2023-05-01|
|  9|2023-03-01|2023-04-01|
+---+----------+----------+

